In [1]:
import pandas as pd
import numpy as np
from collections import Counter, OrderedDict

# Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.


Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).


# Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

Важно:

Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
Если товар не встречался в обучающей выборке, его популярность равна 0.
Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

Задание

1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)

2. Реализуйте два алгоритма рекомендаций:
    * сортировка просмотренных id по популярности (частота появления в просмотренных),
    * сортировка просмотренных id по покупаемости (частота появления в покупках).

3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.


Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)



In [2]:
train = pd.read_csv('coursera_sessions_train.txt', sep=';', header=None, names=['viewed','bought'])
train['viewed'] = train.viewed.str.split(',')
train['bought'] = train.bought.str.split(',')
print('Train data')
print(train.shape)
train.head()

Train data
(50000, 2)


,viewed,bought
0,"[0, 1, 2, 3, 4, 5]",NaN
1,"[9, 10, 11, 9, 11, 12, 9, 11]",NaN
2,"[16, 17, 18, 19, 20, 21]",NaN
3,"[24, 25, 26, 27, 24]",NaN
4,"[34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39]",NaN


In [3]:
test = pd.read_csv('coursera_sessions_test.txt', sep=';', header=None, names=['viewed','bought']).dropna()
test['viewed'] = test.viewed.str.split(',')
test['bought'] = test.bought.str.split(',')
print('Test data')
print(test.shape)
test.head()

Test data
(3665, 2)


,viewed,bought
7,"[63, 68, 69, 70, 66, 61, 59, 61, 66, 68]","[66, 63]"
14,"[158, 159, 160, 159, 161, 162]",[162]
19,"[200, 201, 202, 203, 204]","[201, 205]"
34,"[371, 372, 371]","[371, 373]"
40,[422],[422]


## Сортировка по просмотренным

In [4]:
all_viewed = train.viewed.to_list()

products_viewed = []
for ses in all_viewed:
    for item in ses:
        products_viewed.append(item)
        
freq_dict_v = dict(Counter(products_viewed))

In [5]:
train = train.dropna()

## Сортировка по купленным

In [6]:
all_bought = train.bought.to_list()

products_bought = []
for ses in all_bought:
    for item in ses:
        products_bought.append(item)
        
freq_dict_b = dict(Counter(products_bought))


# Составление рекомендаций

In [55]:
def sort_by_dict(session, sort_dict):
    item_freq = pd.DataFrame(columns=['item','pos','freq'])
    unique = pd.unique(session)
    for i, item in enumerate(unique):
        if item in sort_dict:
            item_freq.loc[i] = [item, i, sort_dict[item]]
            item_freq.sort_values(by=['freq','pos'], ascending=[False,True],inplace=True)
        else:
            item_freq.loc[i] = [item, i , 0]
            item_freq.sort_values(by=['freq','pos'], ascending=[False,True],inplace=True)
#     a = pd.DataFrame(item_freq, columns=['item','pos','freq'])
#     sorted_items = a.sort_values(by=['freq','pos'], ascending=[False,True])['item'].values[:5]
    
    return item_freq.iloc[:,0].values[:5]

In [56]:
%%time
train['recomendation_by_viewed'] = train.viewed.apply(lambda x: sort_by_dict(x, freq_dict_v))
train['recomendation_by_bought'] = train.viewed.apply(lambda x: sort_by_dict(x, freq_dict_b))
train.head()

Wall time: 3min 51s


,viewed,bought,recomendation_by_viewed,recomendation_by_bought,ARec_1b,ARec_1v,APr_1b,APr_1v,ARec_5b,ARec_5v,APr_5b,APr_5v
7,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]","[63, 64, 60, 61, 65]","[60, 63, 67, 59, 61]",0.333333,0.333333,1,1,0.666667,0.666667,0.4,0.4
10,"[84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86]",[86],"[85, 93, 89, 90, 84]","[86, 85, 93, 84, 87]",1,0,1,0,1,1,0.2,0.2
19,"[138, 198, 199, 127]",[199],"[127, 138, 198, 199]","[138, 199, 127, 198]",0,0,0,0,1,1,0.2,0.2
30,"[303, 304, 305, 306, 307, 308, 309, 310, 311, ...",[303],"[303, 306, 304, 307, 309]","[303, 304, 305, 306, 307]",1,1,1,1,1,1,0.2,0.2
33,"[352, 353, 352]",[352],"[352, 353]","[352, 353]",1,1,1,1,1,1,0.2,0.2


In [57]:
%%time
test['recomendation_by_viewed'] = test.viewed.apply(lambda x: sort_by_dict(x, freq_dict_v))
test['recomendation_by_bought'] = test.viewed.apply(lambda x: sort_by_dict(x, freq_dict_b))
test.head()

Wall time: 3min 56s


,viewed,bought,recomendation_by_viewed,recomendation_by_bought,ARec_1b,ARec_1v,APr_1b,APr_1v,ARec_5b,ARec_5v,APr_5b,APr_5v
7,"[63, 68, 69, 70, 66, 61, 59, 61, 66, 68]","[66, 63]","[63, 68, 66, 61, 59]","[63, 68, 69, 70, 66]",0.5,0.5,1,1,1,1,0.4,0.4
14,"[158, 159, 160, 159, 161, 162]",[162],"[158, 162, 160, 159, 161]","[158, 162, 160, 159, 161]",0,0,0,0,1,1,0.2,0.2
19,"[200, 201, 202, 203, 204]","[201, 205]","[204, 202, 203, 200, 201]","[204, 202, 200, 201, 203]",0,0,0,0,0.5,0.5,0.2,0.2
34,"[371, 372, 371]","[371, 373]","[371, 372]","[371, 372]",0.5,0.5,1,1,0.5,0.5,0.2,0.2
40,[422],[422],[422],[422],1,1,1,1,1,1,0.2,0.2


# Расчет метрик

In [58]:
def metrics(rec, true):
    
    APr_1 = len(set(rec[:1]).intersection(true)) 
    ARec_1 = len(set(rec[:1]).intersection(true))/len(true)
    APr_5 = len(set(rec).intersection(true))/5
    ARec_5 = len(set(rec).intersection(true))/len(true) 
    
    return [ARec_1, APr_1, ARec_5, APr_5]

## Подсчет метрик на обучающих данных

In [59]:
%%time
for i, metr in enumerate(['ARec_1', 'APr_1', 'ARec_5', 'APr_5'],0):
    train[metr+'b'] = None
    train[metr+'v'] = None
    for j in range(train.shape[0]):
        train[metr+'b'].iloc[j] = metrics(train['recomendation_by_bought'].iloc[j],
                                          train['bought'].iloc[j])[i]
        train[metr+'v'].iloc[j] = metrics(train['recomendation_by_viewed'].iloc[j],
                                          train['bought'].iloc[j])[i]


Wall time: 5.42 s


## Подсчет метрик на тестовых данных

In [60]:
%%time
for i, metr in enumerate(['ARec_1', 'APr_1', 'ARec_5', 'APr_5'],0):
    test[metr+'b'] = None
    test[metr+'v'] = None
    for j in range(test.shape[0]):
        test[metr+'b'].iloc[j] = metrics(test['recomendation_by_bought'].iloc[j],
                                         test['bought'].iloc[j])[i]
        test[metr+'v'].iloc[j] = metrics(test['recomendation_by_viewed'].iloc[j],
                                         test['bought'].iloc[j])[i]
    

Wall time: 5.12 s


Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой

In [53]:
viewed_alg_train_mean = train[['ARec_1v', 'APr_1v', 'ARec_5v', 'APr_5v']].mean().round(2).values
viewed_alg_test_mean = test[['ARec_1v', 'APr_1v', 'ARec_5v', 'APr_5v']].mean().round(2).values
bought_alg_train_mean = train[['ARec_1b', 'APr_1b', 'ARec_5b', 'APr_5b']].mean().round(2).values
bought_alg_test_mean = test[['ARec_1b', 'APr_1b', 'ARec_5b', 'APr_5b']].mean().round(2).values

In [54]:
print('AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5')
print('сотрировка по просомтренным обуч. данные   ',viewed_alg_train_mean)
print('сотрировка по просомтренным тестовые данные',viewed_alg_test_mean)
print('сотрировка по купленным обуч. данные       ',bought_alg_train_mean)
print('сотрировка по купленным тестовые данные    ',bought_alg_test_mean)

AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5
сотрировка по просомтренным обуч. данные    [0.45 0.52 0.82 0.21]
сотрировка по просомтренным тестовые данные [0.42 0.49 0.82 0.21]
сотрировка по купленным обуч. данные        [0.67 0.78 0.82 0.21]
сотрировка по купленным тестовые данные     [0.47 0.54 0.82 0.21]


In [32]:
def write_answer(x, n):
    with open("answer_{}.txt".format(n), "w") as fout:
        s = ' '.join([str(i) for i in x])
        fout.write(s)

In [33]:
answers = [viewed_alg_train_mean,
           viewed_alg_test_mean,
           bought_alg_train_mean,
           bought_alg_test_mean]
for i in range(len(answers)):
    write_answer(answers[i], i+1)

# Дополнительные вопросы

Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.

Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.